In [55]:
import pandas as pd
import matplotlib
import numpy as np

In [56]:
pd.__version__

'2.1.2'

In [57]:
data_name = "2019-2023-sand500_MinuteToDay_Linear.csv"

In [58]:
data = pd.read_csv("..\..\Data\Day\\" + data_name, index_col = 0)

In [59]:
def data_transformer(data):
    if 'Close' in data.columns:
        data = data.replace(",", "", regex=True)
        data.Date = pd.to_datetime(data.Date)
        data = data.sort_values(by = ["Date"])
        data = data.set_index('Date')
        data.Close = data.Close.astype(float)
        data.High = data.High.astype(float)
        data.Low = data.Low.astype(float)
        data.Open = data.Open.astype(float)
        data["Vol"] = data["Vol"].astype(float)
    else:
        data = data.replace(",", "", regex = True)
        data.Date = pd.to_datetime(data.Date)
        data = data.sort_values(by=["Date"])
        data = data.set_index('Date')
        data.Price = data.Price.astype(float)
        data.High = data.High.astype(float)
        data.Low = data.Low.astype(float)
        data.Open = data.Open.astype(float)
        data["Vol."] = data["Vol."].astype(float)

def to_weeks_transfer(df):
    return df.groupby(df.index.to_period('W')).last()

In [60]:
data_transformer(data)

In [61]:
data.head()

,Date,Open,High,Low,Close,Vol
0,2019-01-02,2510.02,2510.030000,2510.02,2510.030000,2.000000
1,2019-01-03,2447.87,2447.890000,2447.87,2447.890000,3.000000
2,2019-01-04,2531.93,2531.940000,2531.93,2531.940000,2.000000
3,2019-01-05,2537.85,2537.856667,2537.85,2537.856667,1.666667
4,2019-01-06,2543.77,2543.773333,2543.77,2543.773333,1.333333


In [62]:
data['Date'] = pd.to_datetime(data['Date'])

In [63]:
# Устанавливаем столбец 'Date' в качестве индекса
data.set_index('Date', inplace=True)

In [64]:
data

,Open,High,Low,Close,Vol
Date,,,,,
2019-01-02,2510.02,2510.030000,2510.02,2510.030000,2.000000
2019-01-03,2447.87,2447.890000,2447.87,2447.890000,3.000000
2019-01-04,2531.93,2531.940000,2531.93,2531.940000,2.000000
2019-01-05,2537.85,2537.856667,2537.85,2537.856667,1.666667
2019-01-06,2543.77,2543.773333,2543.77,2543.773333,1.333333
...,...,...,...,...,...
2022-12-27,3829.25,3829.250000,3829.25,3829.250000,44.000000
2022-12-28,3783.22,3783.220000,3783.22,3783.220000,47.000000
2022-12-29,3849.28,3849.280000,3849.28,3849.280000,43.000000


In [65]:
if "Vol." in data:
    data = data.drop(['Vol.'], axis = 1)
if 'Change %' in data:
    data = data.drop(['Change %'], axis = 1)

In [66]:
# Проверяем, является ли индекс DataFrame типом DatetimeIndex
if not isinstance(data.index, pd.DatetimeIndex):
    raise TypeError("Index must be DatetimeIndex")

In [67]:
# Группируем данные по неделям и вычисляем среднее значение для каждого столбца
weekly_df = data.resample('W').mean()

In [68]:
# Сбрасываем индекс, чтобы столбец с датами снова стал обычным столбцом
weekly_df.reset_index(inplace=True)

In [69]:
weekly_df

,Date,Open,High,Low,Close,Vol
0,2019-01-06,2514.288000,2514.298000,2514.288000,2514.298000,2.000000
1,2019-01-13,2582.974286,2582.978571,2582.971429,2582.975714,2.571429
2,2019-01-20,2632.814286,2632.815714,2632.674643,2632.677500,6.107143
3,2019-01-27,2647.107143,2647.110000,2647.090357,2647.093214,3.678571
4,2019-02-03,2686.695714,2686.704286,2686.695714,2686.704286,1.857143
...,...,...,...,...,...,...
204,2022-12-04,4036.698571,4036.702857,4036.698571,4036.701429,67.357143
205,2022-12-11,3952.680000,3952.680000,3952.680000,3952.680000,69.428571
206,2022-12-18,3920.135714,3920.150000,3920.131429,3920.145714,58.214286
207,2022-12-25,3838.482857,3838.484286,3838.482857,3838.482857,51.666667


In [70]:
start_date = weekly_df['Date'].iloc[0]

# Создаем новый столбец 'Day' со значениями дней, начиная с 0
weekly_df['Day'] = pd.Series(range(len(data)))

# Создаем новый столбец 'New_Date' с датами, начиная с первого значения даты
weekly_df['New_Date'] = start_date + pd.to_timedelta(weekly_df['Day'], unit='D')

# Заменяем столбец 'Date' столбцом 'New_Date'
weekly_df['Date'] = weekly_df['New_Date']

# Удаляем временные столбцы 'Day' и 'New_Date'
df = weekly_df.drop(['Day', 'New_Date'], axis=1)

In [71]:
df

,Date,Open,High,Low,Close,Vol
0,2019-01-06,2514.288000,2514.298000,2514.288000,2514.298000,2.000000
1,2019-01-07,2582.974286,2582.978571,2582.971429,2582.975714,2.571429
2,2019-01-08,2632.814286,2632.815714,2632.674643,2632.677500,6.107143
3,2019-01-09,2647.107143,2647.110000,2647.090357,2647.093214,3.678571
4,2019-01-10,2686.695714,2686.704286,2686.695714,2686.704286,1.857143
...,...,...,...,...,...,...
204,2019-07-29,4036.698571,4036.702857,4036.698571,4036.701429,67.357143
205,2019-07-30,3952.680000,3952.680000,3952.680000,3952.680000,69.428571
206,2019-07-31,3920.135714,3920.150000,3920.131429,3920.145714,58.214286
207,2019-08-01,3838.482857,3838.484286,3838.482857,3838.482857,51.666667


In [72]:
weekly_df.to_csv("..\..\Data\Week\\" + data_name, index = False)